In [1]:
constellation = {}
constellation_cmplx_numbers = {}

values = [-7, -5, -3, -1, 1, 3, 5, 7]  # Possible values for I and Q components
bit_combinations = [(i, j, k, l, m, n) for i in [0, 1] for j in [0, 1] for k in [0, 1] for l in [0, 1] for m in [0, 1] for n in [0, 1]]

for i, combination in enumerate(bit_combinations):
    I = values[i // 8]  # Integer division to cycle through I values
    Q = values[i % 8]   # Modulus to cycle through Q values
    constellation[combination] = complex(I, Q)

ctr = 0
for k in constellation:
    constellation_cmplx_numbers[constellation[k]] = ctr
    ctr += 1

def cmplx_symb_to_number(symbols):
    out = []
    for smb in symbols:
        out.append(constellation_cmplx_numbers[smb])
    return out
        

In [2]:
import cv2
import numpy as np
import processor as pr

def read_video(vid_path):
    vid_symbols = []
    cap = cv2.VideoCapture(vid_path)
    if not cap.isOpened():
        print("ERROR: READING VIDEO")
    while cap.isOpened():
        ret, frame = cap.read(cv2.IMREAD_GRAYSCALE)
        if len(vid_symbols) > 200: 
            break
        if ret:
            resized_frame = cv2.resize(frame, (128, 96))
            binary_data = np.unpackbits(resized_frame)
            vid_symbols.append(pr.qam64_modulation(binary_data))

        else:
            break
    
    cap.release()
    cv2.destroyAllWindows()
    return vid_symbols

In [3]:
vid1 = read_video("./data/vids/train1.mp4")
vid2 = read_video("./data/vids/train2.mp4")
vid3 = read_video("./data/vids/train3.mp4")
vid4 = read_video("./data/vids/train4.mp4")
print("done")

val1 = read_video("./data/vids/validate1.mp4")
val2 = read_video("./data/vids/validate2.mp4")
print("done")

test1 = read_video("./data/vids/test1.mp4")
test2 = read_video("./data/vids/test2.mp4")
print("done woohoo data")

done
done
done woohoo data


In [ ]:
def build_symbol_np_array(images):
    total_X = np.empty((12, 49152, 1))
    for img in images:
        result_array = []
        for symb in img:
            x, y = symb.real, symb.imag
            result_array.extend([x, y])
        total_X.append(np.array(result_array))
    return np.array(total_X)
        

In [19]:
def data_list(imgs):
    collector = np.empty((3, 49152, 1))
    ct_1 = 0
    for img in imgs:
        img_data = np.empty((49152, 1))
        ct_2 = 0
        for symbol in img:
            img_data[ct_2] = [constellation_cmplx_numbers[symbol]]
            ct_2 += 1
        collector[ct_1] = img_data
        ct_1 += 1
    return collector


In [22]:
#Creating the training dataset
#each data block looks like this [[symbol 1, symbol 2, symbol 3, ...], [...], [...], ...]
train_data_X = np.empty((792, 3, 49152, 1))
train_data_Y = np.empty((792, 49152, 1))
data_index = 0
for i in range(3, len(vid1)):
    train_data_X[data_index] = data_list(vid1[i - 3:i])
    train_data_Y[data_index] = data_list([vid1[i]])[0]
    data_index+=1

    train_data_X[data_index] = data_list(vid2[i - 3:i])
    train_data_Y[data_index] = data_list([vid2[i]])[0]
    data_index+=1

    train_data_X[data_index] = data_list(vid3[i - 3:i])
    train_data_Y[data_index] = data_list([vid3[i]])[0]
    data_index+=1
    #data_list([vid3[i]])[0]

    train_data_X[data_index] = data_list(vid4[i - 3:i])
    train_data_Y[data_index] = data_list([vid4[i]])[0]
    data_index+=1

In [28]:
print(np.shape(train_data_X))
print(np.shape(train_data_Y))

(792, 3, 49152, 1)
(792, 49152, 1)


In [24]:
val__data_index = 0
val_data_X = np.empty((302, 3, 49152, 1))
val_data_Y = np.empty((302, 49152, 1))
for i in range(3, len(val1)):
    if val__data_index > 300:
        break
    val_data_X[val__data_index] = data_list(val1[i - 3:i])
    val_data_Y[val__data_index] = data_list([val1[i]])[0]
    val__data_index+=1

    val_data_X[val__data_index] = data_list(val2[i - 3:i])
    val_data_Y[val__data_index] = data_list([val2[i]])[0]
    val__data_index+=1

In [ ]:
print(np.shape(val_data_X))
print(np.shape(val_data_Y))

In [ ]:
del(vid1, vid2, vid3, vid4, val1, val2)

In [ ]:
test_data_index = 0
test_data_X = np.empty((302, 3, 49152))
test_data_Y = np.empty((302, 49152))
for i in range(3, len(test1)):
    if test_data_index > 300:
        break
    test_data_X[test_data_index, :] = data_list(test1[i - 3:i])
    test_data_Y[test_data_index] = data_list([test1[i]])[0]
    test_data_index+=1

    test_data_X[test_data_index, :] = data_list(test2[i - 3:i])
    test_data_Y[test_data_index] = data_list([test2[i]])[0]
    test_data_index+=1

In [ ]:
print(test_data_X)

In [ ]:
#Simple RNN
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, LSTM

# Define the model
model = Sequential([
    # SimpleRNN layer with 256 units
    SimpleRNN(256, input_shape=(12, 49152), return_sequences = True, activation='relu'),
    
    # You can add more SimpleRNN layers if needed, setting return_sequences=True in the previous layers
    SimpleRNN(128, return_sequences=False, activation='relu'),
    
    # Final Dense layer to output 125 symbols
    Dense(49152, activation='linear')  # 'linear' activation for regression-like tasks
])

# Compile the model
model.compile(optimizer='adam', loss='mse')  # Mean Squared Error for regression-like tasks

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Reshape

# Input shape parameters
num_images = 3
num_symbols_per_image = 49152

model = Sequential()

# Reshape input to flatten the symbols in each image while keeping the image sequence
model.add(Reshape((num_images, -1), input_shape=(num_images, num_symbols_per_image, 1)))

# LSTM layer to process the sequence of images
# You might need to adjust the number of units
model.add(LSTM(256, return_sequences=True))

model.add(LSTM(256, return_sequences=True))

model.add(LSTM(256, return_sequences=False))

# Output layer: since the output is a single image represented as 49152 symbols,
# and each symbol is one of 64 classes, use TimeDistributed with Dense 64
model.add(Dense(num_symbols_per_image * 64))
model.add(Reshape((num_symbols_per_image, 64)))

model.add(TimeDistributed(Dense(64, activation='softmax')))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [33]:
train_data_X_res = np.transpose(np.squeeze(train_data_X, axis=-1), (0, 2, 1))
train_data_Y_res = np.squeeze(train_data_Y, axis=-1)

val_data_X_res = np.transpose(np.squeeze(val_data_X, axis=-1), (0, 2, 1))
val_data_Y_res = np.squeeze(val_data_Y, axis=-1)


In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense

# Model parameters
num_filters = 32      # Number of convolutional filters
kernel_size = 3       # Size of the convolutional kernel
pool_size = 2         # Size of the pooling window
lstm_units = 128      # Number of units in the LSTM layer

model = Sequential()

# 1D Convolutional layers
model.add(Conv1D(filters=num_filters, kernel_size=kernel_size, activation='relu', input_shape=(49152, 3)))
model.add(MaxPooling1D(pool_size=pool_size))

# Optional: more Conv1D layers
# model.add(Conv1D(filters=num_filters, kernel_size=kernel_size, activation='relu'))
# model.add(MaxPooling1D(pool_size=pool_size))

# LSTM layer
# Before adding LSTM layer, you might need to flatten or reshape depending on your data and the preceding layers
model.add(LSTM(lstm_units))

# Output layer: Adjust the units and activation according to your problem
model.add(Dense(49152 * 64))
model.add(Reshape((49152, 64)))

model.add(TimeDistributed(Dense(64, activation='softmax')))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [40]:
history = model.fit(train_data_X_res, train_data_Y_res, epochs=5, batch_size=64, validation_data=(val_data_X_res, val_data_Y_res))

test_loss = model.evaluate(test_data_X, test_data_Y)

Epoch 1/5
13/13 [==============================] - 552s 41s/step - loss: 1.6503 - accuracy: 0.4621 - val_loss: 6.3106 - val_accuracy: 0.0622
Epoch 2/5
13/13 [==============================] - 487s 38s/step - loss: 1.6336 - accuracy: 0.4674 - val_loss: 6.2277 - val_accuracy: 0.0625
Epoch 3/5
13/13 [==============================] - 567s 44s/step - loss: 1.6184 - accuracy: 0.4715 - val_loss: 6.4142 - val_accuracy: 0.0622
Epoch 4/5
13/13 [==============================] - 549s 42s/step - loss: 1.6009 - accuracy: 0.4774 - val_loss: 6.0294 - val_accuracy: 0.0629
Epoch 5/5
13/13 [==============================] - 594s 46s/step - loss: 1.5880 - accuracy: 0.4809 - val_loss: 6.2921 - val_accuracy: 0.0625


NameError: name 'test_data_X' is not defined

In [ ]:
data = train_data_X[622]
data = data.reshape(1, 12, 49152)
print(np.shape(data))
model.predict(data)

In [39]:
model.save("fancy_layer.keras")

In [6]:
import tensorflow as tf
model = tf.keras.models.load_model("3_layer_RNN1.keras")

In [ ]:
img_np_data = np.empty((1, 12, 49152))
img_np_data[0, :] = data_list(test1[0:12])
x  = model.predict(img_np_data)
round(26.5)


In [ ]:
print(test1[11])

In [41]:
# Now do the data parsing and test accuracy vs stat methods
# We will use test 1 that we used for testing. 
import stat_model as stm
import processor as pr
import importlib

importlib.reload(stm)
importlib.reload(pr)

expected_LRU = vid1[2]
expected_PROB = vid1[2]
qam64_const = list(constellation.values())

ct = 0
no_total = 0
LRU_total = 0
PROB_total = 0
ml_total = 0

for i in range(3, len(vid1)):
    img = pr.add_noise(vid1[i], 15)
    actual_image = stm.process_simple(vid1[i], qam64_const)
    #process with no tools
    no_processing_decoding = stm.process_simple(img, qam64_const)
    
    #process with stat_models
    expected_LRU, LRU_decoding = stm.process_QPSK_image_stream_LRU(img, expected_LRU, qam64_const)
    expected_PROB, PROB_decoding = stm.process_QPSK_image_stream_PROB(img, expected_PROB, qam64_const, 0.5)

    #process with ML model
    img_np_data = np.empty((1, 3, 49152, 1))
    img_np_data[0, :] = data_list(vid1[i - 3:i])
    
    ml_pred = model.predict(img_np_data)
    ml_decoding = pr.process_ML_model_pred(img, np.argmax(ml_pred, axis=-1), qam64_const)

    #compare different demodulations
    no_total += pr.symbol_stream_diff(actual_image, no_processing_decoding)
    LRU_total += pr.symbol_stream_diff(actual_image, LRU_decoding)
    PROB_total += pr.symbol_stream_diff(actual_image, PROB_decoding)
    ml_total += pr.symbol_stream_diff(actual_image, ml_decoding)
    ct+=1

print(no_total/ct)
print(LRU_total/ct)
print(PROB_total/ct)
print(ml_total/ct)
    

ValueError: in user code:

    File "/Users/andriiiermolaiev/Library/Python/3.9/lib/python/site-packages/keras/src/engine/training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "/Users/andriiiermolaiev/Library/Python/3.9/lib/python/site-packages/keras/src/engine/training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/andriiiermolaiev/Library/Python/3.9/lib/python/site-packages/keras/src/engine/training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/andriiiermolaiev/Library/Python/3.9/lib/python/site-packages/keras/src/engine/training.py", line 2381, in predict_step
        return self(x, training=False)
    File "/Users/andriiiermolaiev/Library/Python/3.9/lib/python/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/andrii